In [7]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
from torchvision import datasets, transforms
from sklearn.model_selection import StratifiedShuffleSplit
from PIL import Image

In [2]:
data_dir='data/animals'

#加载完整的数据集
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
full_dataset = datasets.ImageFolder(root=data_dir,transform=transform)

#从数据集中提取标签
labels = np.array([label for _, label in full_dataset.samples])

#定义分割数据集的参数
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

#生成训练集和测试集的索引
train_idx, test_idx = next(sss.split(np.zeros(len(labels)), labels))

# 创建用于训练和测试的子集
train_dataset = Subset(full_dataset, train_idx)
test_dataset = Subset(full_dataset, test_idx)

# 为训练和测试数据集创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False,num_workers=4)


In [ ]:
# 加载预训练好的ResNet-50
resnet50 = models.resnet50(pretrained=True)

# 为了进行第一步迁移学习，首先锁定模型的参数
for param in resnet50.parameters():
    param.requires_grad = False

# 数据集有90个分类，将最后一层全连接层的输出值置为90
resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, 90)

# 定义损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
# 只优化最后一层
optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=0.0005)

# 使用GPU训练
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)
resnet50 = torch.nn.DataParallel(resnet50)

# 将模型设置为训练模式
resnet50.train()

# 迁移学习，这一阶段只调整最后一层全连接层的参数
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 将参数梯度归零
        optimizer.zero_grad()

        # 前向传播
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        # 反向传播并优化参数
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(
        f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# 允许对整个模型进行微调
for param in resnet50.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.0001)
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 将参数梯度归零
        optimizer.zero_grad()

        # 前向传播
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        # 反向传播并优化参数
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(
        f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# 将模型设置为评估模式
resnet50.eval()
# 保存模型
torch.save(resnet50.state_dict(), 'resnet50_state_dict.pth')
print("Done")

In [8]:
# 加载模型结构
model = models.resnet50()
model.fc = torch.nn.Linear(model.fc.in_features, 90)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load('resnet50_state_dict.pth'))
model.eval()

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 打开标签
with open('labels.txt', 'r', encoding='utf-8') as file:
    labels = file.readlines()
labels = [label.strip() for label in labels]

In [9]:
# 加载图像并进行预处理
image_path = r"../public/images/upload.jpg"
image = Image.open(image_path).convert('RGB')

input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # 维度扩展

# 将模型移动到相同的设备上并进行推理
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_batch = input_batch.to(device)
model = model.to(device)

# 进行预测
with torch.no_grad():
    output = model(input_batch)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

top10_prob, top10_indices = torch.topk(probabilities, 10)
for i in range(len(top10_prob)):
    label = labels[top10_indices[i]]
    prob = top10_prob[i].item()
    print(f"标签: {label}, 概率: {prob:.4f}")

标签: butterfly, 概率: 0.5419
标签: hornbill, 概率: 0.3694
标签: dolphin, 概率: 0.0107
标签: caterpillar, 概率: 0.0106
标签: okapi, 概率: 0.0084
标签: penguin, 概率: 0.0076
标签: chimpanzee, 概率: 0.0064
标签: lobster, 概率: 0.0060
标签: bat, 概率: 0.0060
标签: woodpecker, 概率: 0.0051
